In [ ]:
import numpy as np
import sympy as sym
from scipy import linalg
from scipy import signal

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

# Is the reaction wheel controllable?

Parameters:

In [ ]:
# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

Equations of motion:

$$ J_w \dot{v} = \tau $$

In [ ]:
# Convert floats to rationals
Jw_sym = sym.nsimplify(Jw)

# Create symbolic variables
v, tau = sym.symbols('v, tau', real=True)

# Define equations of motion
f = sym.Matrix([[(1 / Jw_sym) * tau]])

# Show the result
f

Linearization:

In [ ]:
ve = 1. * (2 * np.pi) # <-- 1 rpm
taue = 0.

A_num = sym.lambdify((v, tau), f.jacobian([v]))
B_num = sym.lambdify((v, tau), f.jacobian([tau]))
A = A_num(ve, taue).astype(float)
B = B_num(ve, taue).astype(float)

In [ ]:
print(A)

In [ ]:
print(B)

Controllability matrix:

In [ ]:
W = np.block([[B]])

In [ ]:
print(W)

Rank of controllability matrix:

In [ ]:
np.linalg.matrix_rank(W)

Number of states:

In [ ]:
A.shape[0]

What happens if you try to do eigenvalue placement?

In [ ]:
fbk = signal.place_poles(A, B, [-1.])
K = fbk.gain_matrix

Did it work?

In [ ]:
print(linalg.eigvals(A - B @ K))

# Is the platform controllable, ignoring the reaction wheel?

Parameters:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

Equations of motion:

$$ (J_p + ml^2) \ddot{q} = \tau $$

In [ ]:
# Convert floats to rationals
l_sym = sym.nsimplify(l)
mw_sym = sym.nsimplify(mw)
Jp_sym = sym.nsimplify(Jp)

# Create symbolic variables
q, v, tau = sym.symbols('q, v, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v], [(1 / (Jp_sym + mw_sym * l_sym**2)) * tau]])

# Show the result
f

Linearization:

In [ ]:
qe = 0.
ve = 0.
taue = 0.

A_num = sym.lambdify((q, v, tau), f.jacobian([q, v]))
B_num = sym.lambdify((q, v, tau), f.jacobian([tau]))
A = A_num(qe, ve, taue).astype(float)
B = B_num(qe, ve, taue).astype(float)

In [ ]:
print(A)

In [ ]:
print(B)

Controllability matrix:

In [ ]:
W = np.block([[B, A @ B]])

In [ ]:
print(W)

Rank of controllability matrix:

In [ ]:
np.linalg.matrix_rank(W)

Number of states:

In [ ]:
A.shape[0]

What happens if you try to do eigenvalue placement?

In [ ]:
fbk = signal.place_poles(A, B, [-1., -2.])
K = fbk.gain_matrix

Did it work?

In [ ]:
print(linalg.eigvals(A - B @ K))

# Is the platform controllable, *not* ignoring the reaction wheel?

Parameters:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [ ]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1], [(1 / J1) * tau], [(1 / J2) * (- ((J1 + J2) / J1) * tau)]])

# Show the result
f

Linearization:

In [ ]:
q1e = 0.
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [ ]:
print(A)

In [ ]:
print(B)

Controllability matrix:

In [ ]:
W = np.block([[B, A @ B, A @ A @ B]])

In [ ]:
print(W)

Rank of controllability matrix:

In [ ]:
np.linalg.matrix_rank(W)

Number of states:

In [ ]:
A.shape[0]

What happens if you try to do eigenvalue placement?

In [ ]:
fbk = signal.place_poles(A, B, [-1., -2., -3.])
K = fbk.gain_matrix

Did it work?

In [ ]:
print(linalg.eigvals(A - B @ K))

# What if there is damping?

Parameters:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# coefficient of viscous friction
c1 = 0.1
c2 = 0.1

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - c_1 v_1 + c_2 v_2 \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + c_1 \left(\dfrac{J_2}{J_1}\right) v_1 - c_2 \left( \dfrac{J_1 + J_2}{J_1} \right) v_2
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [ ]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)
c1 = sym.nsimplify(c1)
c2 = sym.nsimplify(c2)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1],
                [(1 / J1) * (tau - c1 * v1 + c2 * v2)],
                [(1 / J2) * (- ((J1 + J2) / J1) * tau + c1 * (J2 / J1) * v1 - c2 * ((J1 + J2) / J1) * v2)]])

# Show the result
f

Linearization:

In [ ]:
q1e = 0.
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [ ]:
print(A)

In [ ]:
print(B)

Controllability matrix:

In [ ]:
W = np.block([[B, A @ B, A @ A @ B]])

In [ ]:
print(W)

Rank of controllability matrix:

In [ ]:
np.linalg.matrix_rank(W)

Number of states:

In [ ]:
A.shape[0]

What happens if you try to do eigenvalue placement?

In [ ]:
fbk = signal.place_poles(A, B, [-1., -2., -3.])
K = fbk.gain_matrix

Did it work?

In [ ]:
print(linalg.eigvals(A - B @ K))

# What if there is gravity (i.e., non-zero ground pitch)?

Parameters:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

Equations of motion:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)
\end{aligned}
$$

where

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

In [ ]:
# Convert floats to rationals
J1 = sym.nsimplify(Jp + mw * l**2)
J2 = sym.nsimplify(Jw)
mw = sym.nsimplify(mw)
g = sym.nsimplify(g)
l = sym.nsimplify(l)

# Create symbolic variables
q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau', real=True)

# Define equations of motion
f = sym.Matrix([[v1], [(1 / J1) * (tau - mw * g * l * sym.sin(q1))], [(1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))]])

# Show the result
f

Linearization:

Nnote that $\tau=0$ at any equilibrium point — check this yourself! — and so there are exact two choices for $q_1$:

$$q_1 = 0 \qquad\text{or}\qquad q_1 = \pi$$

In [ ]:
q1e = np.pi
v1e = 0.
v2e = 0.
taue = 0.

A_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([q1, v1, v2]))
B_num = sym.lambdify((q1, v1, v2, tau), f.jacobian([tau]))
A = A_num(q1e, v1e, v2e, taue).astype(float)
B = B_num(q1e, v1e, v2e, taue).astype(float)

In [ ]:
print(A)

In [ ]:
print(B)

Controllability matrix:

In [ ]:
W = np.block([[B, A @ B, A @ A @ B]])

In [ ]:
print(W)

Rank of controllability matrix:

In [ ]:
np.linalg.matrix_rank(W)

Number of states:

In [ ]:
A.shape[0]

What happens if you try to do eigenvalue placement?

In [ ]:
fbk = signal.place_poles(A, B, [-1., -2., -3.])
K = fbk.gain_matrix

Did it work?

In [ ]:
print(linalg.eigvals(A - B @ K))

Print $K$ for copy/paste into a controller:

In [ ]:
print(K.tolist())

Do you think all those significant digits really matter?

In [ ]:
K = K.round(decimals=4)
print(K.tolist())

What happened to the eigenvalues?

In [ ]:
print(linalg.eigvals(A - B @ K))

# Questions to consider

* What if you want to control the *angle* of the wheel and not just the angular velocity, with gravity?
* What if there is only damping at one joint and not the other?
* What if there is both gravity and damping?
* What if you change the physical parameters (masses, moments of inertia, etc.)?
* Etc.